In [1]:
import cv2
import imutils
import numpy as np
import pytesseract
# to compute angles
from math import atan2, cos, sin, sqrt, pi
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
from skimage.filters import threshold_local
import tensorflow as tf
from skimage import measure

In [2]:
# Referencia https://www.geeksforgeeks.org/detect-and-recognize-car-license-plate-from-a-video-in-real-time/

def clean_plate(self, plate):
 
    gray = cv2.cvtColor(plate, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255,
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)
     
    contours, _ = cv2.findContours(thresh.copy(),
                                      cv2.RETR_EXTERNAL,
                                      cv2.CHAIN_APPROX_NONE)
 
    if contours:
         
        areas = [cv2.contourArea(c) for c in contours]
         
        # index of the largest contour in the
        # areas array
        max_index = np.argmax(areas) 
 
        max_cnt = contours[max_index]
        max_cntArea = areas[max_index]
        x, y, w, h = cv2.boundingRect(max_cnt)
 
        if not self.ratioCheck(max_cntArea,
                               plate.shape[1],
                               plate.shape[0]):
            return plate, False, None
         
        return plate, True, [x, y, w, h]
     
    else:
        return plate, False, None
    
def ratioCheck(self, area, width, height):
     
    min = self.min_area
    max = self.max_area
 
    ratioMin = 3
    ratioMax = 6
 
    ratio = float(width) / float(height)
     
    if ratio < 1:
        ratio = 1 / ratio
 
    if (area < min or area > max) or (ratio < ratioMin or ratio > ratioMax):
        return False
     
    return True

In [19]:

for i in range(1, 20):

    img    = cv2.imread(f"archive/images/Cars{i}.png")
    # cv2.imshow('HelloWorld', img)
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(grey, 13, 15, 15) 

    edged = cv2.Canny(gray, 30, 200) 
    contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    screenCnt = None


    for c in contours:
        
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * peri, True)

        if len(approx) == 4:
            screenCnt = approx
            break

    if screenCnt is None:
        detected = 0
        print ("No contour detected")
        continue
        
    else:
        detected = 1

    if detected == 1:
        cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

    mask = np.zeros(gray.shape,np.uint8)
    new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
    new_image = cv2.bitwise_and(img,img,mask=mask)

No contour detected
No contour detected
No contour detected
No contour detected
No contour detected
No contour detected
No contour detected


In [5]:
import time
import math

cap = cv2.VideoCapture('Traffic_Camera.avi')

frameRate = cap.get(5) #frame rate

x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename = './test_images/image' +  str(int(x)) + ".jpg";x+=1
        cv2.imwrite(filename, frame)
cap.release()

In [21]:
for i in range(1, 30):

    img = cv2.imread(f"test_images/image{i}.jpg")
    y=0
    x=1400
    h=700
    w=500
    img = img[y:y+h, x:x+w]
#     cv2.imshow('car',img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(grey, 13, 15, 15) 

    edged = cv2.Canny(gray, 30, 200) 
    contours = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:10]
    screenCnt = None

    for c in contours:
        
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.018 * peri, True)

        if len(approx) == 4:
            screenCnt = approx
            break

    if screenCnt is None:
        detected = 0
        print ("No contour detected")
        continue
        
    else:
        detected = 1

    if detected == 1:
        cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

    mask = np.zeros(gray.shape,np.uint8)
    new_image = cv2.drawContours(mask,[screenCnt],0,255,-1,)
    new_image = cv2.bitwise_and(img,img,mask=mask)
    (x, y) = np.where(mask == 255)
    (topx, topy) = (np.min(x), np.min(y))
    (bottomx, bottomy) = (np.max(x), np.max(y))
    Cropped = gray[topx:bottomx+1, topy:bottomy+1]

    text = pytesseract.image_to_string(Cropped)
    print("programming_fever's License Plate Recognition\n")
    print("Detected license plate Number is:",text)
    img = cv2.resize(img,(500,700))
    Cropped = cv2.resize(Cropped,(400,200))
#     cv2.imshow('car',img)
#     cv2.imshow('Cropped',Cropped)

#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    

programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
programming_fever's License Plate Recognition

Detected license plate Number is: 
prog

In [3]:
def draw_axis(img, p_, q_, colour, scale):
    p = list(p_)
    q = list(q_)
    angle = atan2(p[1] - q[1], p[0] - q[0]) # angle in radians
    hypotenuse = sqrt((p[1] - q[1]) * (p[1] - q[1]) + (p[0] - q[0]) * (p[0] - q[0]))
    # Here we lengthen the arrow by a factor of scale
    q[0] = p[0] - scale * hypotenuse * cos(angle)
    q[1] = p[1] - scale * hypotenuse * sin(angle)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    # create the arrow hooks
    p[0] = q[0] + 9 * cos(angle + pi / 4)
    p[1] = q[1] + 9 * sin(angle + pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    p[0] = q[0] + 9 * cos(angle - pi / 4)
    p[1] = q[1] + 9 * sin(angle - pi / 4)
    cv2.line(img, (int(p[0]), int(p[1])), (int(q[0]), int(q[1])), colour, 1, cv2.LINE_AA)
    
def get_orientation(pts, img):
    sz = len(pts)
    data_pts = np.empty((sz, 2), dtype=np.float64)
    for i in range(data_pts.shape[0]):
        data_pts[i,0] = pts[i,0,0]
        data_pts[i,1] = pts[i,0,1]
    # Perform PCA analysis
    mean = np.empty((0))
    mean, eigenvectors, eigenvalues = cv2.PCACompute2(data_pts, mean)
    # Store the center of the object
    cntr = (int(mean[0,0]), int(mean[0,1]))
    cv2.circle(img, cntr, 3, (255, 0, 255), 2)
    p1 = (cntr[0] + 0.02 * eigenvectors[0,0] * eigenvalues[0,0], cntr[1] + 0.02 *  eigenvectors[0,1] * eigenvalues[0,0])
    p2 = (cntr[0] - 0.02 * eigenvectors[1,0] * eigenvalues[1,0], cntr[1] - 0.02 * eigenvectors[1,1] * eigenvalues[1,0])
    draw_axis(img, cntr, p1, (0, 150, 0), 1)
    draw_axis(img, cntr, p2, (200, 150, 0), 5)
    angle = atan2(eigenvectors[0,1], eigenvectors[0,0]) # orientation in radians
    return angle

In [4]:
for c in contours:
    
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.018 * peri, True)

    if len(approx) == 4:
        screenCnt = approx
        break


if screenCnt is None:
    detected = 0
    print ("No contour detected")
    
    
else:
    detected = 1

if detected == 1:
    cv2.drawContours(img, [screenCnt], -1, (0, 0, 255), 3)

get_orientation(c,img)
cv2.imshow('frame',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

NameError: name 'contours' is not defined